In [1]:
import zipfile
from pathlib import Path
from tempfile import TemporaryFile
from selenium.webdriver import Chrome, ChromeService
import requests

MAJOR_VERSION = 141
PLATFORM = "mac-x64"
DIRECTORY = Path(".")
CHUNK = 8192
LABS = "https://googlechromelabs.github.io/chrome-for-testing/LATEST_RELEASE_{}"
APIS = "https://storage.googleapis.com/chrome-for-testing-public/{}/{}/chromedriver-{}.zip"


def get_driver():
    """
    Determine most up-to-date version of the specified major version
    Download and extract the relevant archive
    """
    with requests.Session() as session:
        # find the most up-to-date version for the specified major version
        with session.get(LABS.format(MAJOR_VERSION)) as response:
            response.raise_for_status()
            version = response.text.strip()
            url = APIS.format(version, PLATFORM, PLATFORM)
            # download the zip archive that contains the driver
            with session.get(url, stream=True) as response:
                response.raise_for_status()
                with TemporaryFile() as fd:
                    for chunk in response.iter_content(CHUNK):
                        fd.write(chunk)
                    fd.flush()
                    DIRECTORY.mkdir(parents=True, exist_ok=True)
                    with zipfile.ZipFile(fd) as z:
                        z.extractall(DIRECTORY)


if __name__ == "__main__":
    driver = DIRECTORY / f"chromedriver-{PLATFORM}" / "chromedriver"

    if not driver.exists():
        get_driver()
        driver.chmod(0o755)

    service = ChromeService(executable_path=driver)

    with Chrome(service=service) as chrome:
        chrome.get("https://www.google.com")
        input("Press return: ")